### Hyper parameter tuning

In [1]:
import pandas as pd

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
from sklearn.datasets import load_iris
iris = load_iris()

In [4]:
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df["target"] = iris.target
df["flower name"] = df.target.apply(lambda x : iris.target_names[x])
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,flower name
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica
149,5.9,3.0,5.1,1.8,2,virginica


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.2)

In [12]:
model = SVC(max_iter=10000)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9666666666666667

In [13]:
from sklearn.model_selection import cross_val_score

In [16]:
import numpy as np

In [18]:
np.average(cross_val_score(SVC(max_iter=10000, C=1.0, kernel='rbf', degree=3,gamma='scale'), iris.data, iris.target))

0.9666666666666666

In [19]:
np.average(cross_val_score(SVC(max_iter=10000, C=10, kernel='linear', degree=3,gamma='auto'), iris.data, iris.target))

0.9733333333333334

In [20]:
np.average(cross_val_score(SVC(max_iter=10000, C=30, kernel='rbf', degree=3,gamma='scale'), iris.data, iris.target))

0.9733333333333334

In [22]:
np.average(cross_val_score(SVC(max_iter=10000, C=30, kernel='linear', degree=3,gamma='auto'), iris.data, iris.target, cv = 5))

0.96

In [29]:
kernel = ["linear", "rbf"]
C = [1, 20, 30]
gamma = ["auto", "scale"]
average_scores = {}
for knl in kernel:
    for cl in C:
        for g in gamma:
            crossval = cross_val_score(SVC(max_iter=10000, kernel = knl, C = cl, gamma=g),iris.data, iris.target, cv = 5)
            average_scores[knl + '_' + str(cl)+ '_'+ g] = np.average(crossval)

In [30]:
average_scores

{'linear_1_auto': 0.9800000000000001,
 'linear_1_scale': 0.9800000000000001,
 'linear_20_auto': 0.9666666666666666,
 'linear_20_scale': 0.9666666666666666,
 'linear_30_auto': 0.96,
 'linear_30_scale': 0.96,
 'rbf_1_auto': 0.9800000000000001,
 'rbf_1_scale': 0.9666666666666666,
 'rbf_20_auto': 0.9666666666666668,
 'rbf_20_scale': 0.9800000000000001,
 'rbf_30_auto': 0.96,
 'rbf_30_scale': 0.9733333333333334}

### GridSearchCV

In [38]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(SVC(max_iter=10000),{
    'kernel' : ["linear", "rbf"],
    'C' : [1, 20, 30]
}, cv =5  )

In [39]:
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00319228, 0.00059962, 0.00079961, 0.00059938, 0.00059962,
        0.00090694]),
 'std_fit_time': array([0.00490105, 0.00048959, 0.0003998 , 0.0004894 , 0.00048959,
        0.00049903]),
 'mean_score_time': array([0.00179939, 0.00039968, 0.00039949, 0.00059981, 0.00039978,
        0.00039968]),
 'std_score_time': array([0.00159917, 0.00048951, 0.00048928, 0.00048975, 0.00048963,
        0.00048951]),
 'param_C': masked_array(data=[1, 1, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 30, 'kernel': 'linear'},
  {'C': 30, 'kernel': 'rbf'}],


In [40]:
df = pd.DataFrame(clf.cv_results_)

In [41]:
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003192,0.004901,0.001799,0.001599,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000600,0.000490,0.000400,0.000490,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,0.966667,0.966667,0.933333,1.0,0.966667,0.021082,4
2,0.000800,0.000400,0.000399,0.000489,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,4
3,0.000599,0.000489,0.000600,0.000490,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.000600,0.000490,0.000400,0.000490,30,linear,"{'C': 30, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.900000,1.0,0.960000,0.048990,6
5,0.000907,0.000499,0.000400,0.000490,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.966667,1.000000,0.933333,0.966667,1.0,0.973333,0.024944,3


In [44]:
df[["params", "mean_test_score", "rank_test_score"]]

,params,mean_test_score,rank_test_score
0,"{'C': 1, 'kernel': 'linear'}",0.980000,1
1,"{'C': 1, 'kernel': 'rbf'}",0.966667,4
2,"{'C': 20, 'kernel': 'linear'}",0.966667,4
3,"{'C': 20, 'kernel': 'rbf'}",0.980000,1
4,"{'C': 30, 'kernel': 'linear'}",0.960000,6
5,"{'C': 30, 'kernel': 'rbf'}",0.973333,3


### RandomizedSearchCV

In [45]:
from sklearn.model_selection import RandomizedSearchCV

rf = RandomizedSearchCV(SVC(max_iter=10000),{
    'kernel' : ["linear", "rbf"],
    'C' : [1, 20, 30]
    },
    cv =5,
    n_iter=2
)    

In [47]:
rf.fit(iris.data, iris.target)
df = pd.DataFrame(rf.cv_results_)

df[["params", "mean_test_score", "rank_test_score"]]

,params,mean_test_score,rank_test_score
0,"{'kernel': 'linear', 'C': 1}",0.98,1
1,"{'kernel': 'linear', 'C': 30}",0.96,2


### Choosing the best model to use

In [48]:
model_params = {
    'svm': {
        'model': SVC(max_iter=10000),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(max_iter=10000),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [49]:

scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.960000,{'n_estimators': 1}
2,logistic_regression,0.973333,{'C': 1}


#### Based on above, I can conclude that SVM with C=1 and kernel='linear' is the best model for solving my problem of iris flower classification